In [1]:
import Pkg 
Pkg.activate(@__DIR__)
Pkg.resolve()
Pkg.instantiate()

import FiniteDiff
import ForwardDiff as FD
import Convex as cvx 
import ECOS
import MuJoCo as MJ

using LinearAlgebra
import Plots
using Random
using JLD2
using Test
using StaticArrays
using Printf
using Distributions
using MathOptInterface
using MuJoCo

  Activating project at `~/daniel/ocrl-piano/single_finger_free_trajectories`
  No Changes to `~/daniel/ocrl-piano/single_finger_free_trajectories/Project.toml`
  No Changes to `~/daniel/ocrl-piano/single_finger_free_trajectories/Manifest.toml`


In [2]:
Threads.nthreads()
install_visualiser()
init_visualiser()

   Resolving package versions...
  No Changes to `~/daniel/ocrl-piano/single_finger_free_trajectories/Project.toml`
  No Changes to `~/daniel/ocrl-piano/single_finger_free_trajectories/Manifest.toml`
┌ Warning: The visualiser for MuJoCo.jl is designed to run multi-threaded, but the current Julia session was started with only one thread. Degraded performance will occur. To enable multi-threading, set JULIA_NUM_THREADS to a value greater than 1 before starting Julia.
└ @ VisualiserExt /home/workstation4/.julia/packages/MuJoCo/9JERS/ext/VisualiserExt/VisualiserExt.jl:76


In [3]:
# model = load_model("models/scene_right_piano_hand.xml")
model = load_model("/home/workstation4/daniel/ocrl-piano/Julia_fmincon_tests/piano_with_one_shadow_hand/scene.xml")
data = init_data(model)
# reset!(model, data)
# visualise!(model, data)

# MJ.site(data, "fingertip_index")
# MJ.site(data, "fingertip_thumb")

MJ.site(data, "rh_shadow_hand/ffdistal_touch_site")
# MJ.site(data, "piano/white_key_site_71")


DataSite:
id:    97
name:  "rh_shadow_hand/ffdistal_touch_site"
xpos:  (3 x  )	Float64	[0.0, 0.0, 0.0] (4 s.f.)
xmat:  (9 x  )	Float64	[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] (4 s.f.)


In [4]:
fingertip = MJ.site(data, "fingertip_index")
fingertip.xpos

target = MJ.geom(data, "keymarker_1")
target.xpos

error = zeros(3)
println(error)
error = fingertip.xpos - target.xpos
println(error)

BoundsError: BoundsError: attempt to access 124×3 transpose(::UnsafeArrays.UnsafeArray{Float64, 2}) with eltype Float64 at index [0, 1:3]

In [5]:
# function multifinger_IK_cost(m::Model, d::Data, neutral_pose::Vector, fingertip_targets::Vector)

# end

function multifinger_IK_cost(params::NamedTuple, x::Vector)::Real
    """
    Compute the cost of the IK problem for a given set of joint angles q and target names.
    """
    model = params.model
    data = params.data
    neutral_joint_angles = params.neutral_joint_angles
    fingertip_targets = params.fingertip_site_names
    qpos = params.neutral_joint_angles
    Q = params.Q

    cost = 0.0
    cost += 0.5*(x - neutral_joint_angles)'*Q*(x - neutral_joint_angles) # regularization cost

    data.qpos .= x
    # step!(model, data)

    return cost
end

function IK_equality_constraints(params::NamedTuple, x::Vector)::Vector
    """
    Compute the equality constraints for the IK problem for a given set of joint angles q and target names.
    """
    model = params.model
    data = params.data
    neutral_joint_angles = params.neutral_joint_angles
    fingertip_site_names = params.fingertip_site_names
    target_site_names = params.target_site_names
    neutral_joint_angles = params.neutral_joint_angles

    data.qpos .= x
    step!(model, data)

    @assert length(x) == length(params.neutral_joint_angles)
    @assert length(params.fingertip_site_names) == length(params.target_site_names)


    error = zeros(length(fingertip_site_names)*3) # positional error for fingertips 
    
    for i=1:length(fingertip_site_names)
        fingertip = MJ.site(data, fingertip_site_names[i])
        # target = MJ.geom(data, target_site_names[i])
        target = MJ.site(data, target_site_names[i])

        # site_id = mj_name2id(model, mjOBJ_SITE, fingertip_site_names[i])
        # target_id = mj_name2id(model, mjOBJ_SITE, target_site_names[i])
        
        # preallocate arrays for a single site computation
        # error_pos = error[1+3*(i-1):3*i]
        # error_rot = error[6*i+3:6*(i+1)]
        
        # positional error
        error[1+3*(i-1):3*i] .= fingertip.xpos - target.xpos



        # # orientation error
        # site_quat = neros(4)
        # site_quat_conj = zeros(4)
        
        # target_quat = zeros(4)
        
        # error_quat = zeros(4)
        
        # mujoco.mju_mat2Quat(site_quat, data.site(site_id).xmat)
        # mujoco.mju_mat2Quat(target_quat, data.geom(target_id).xmat)
        
        # mujoco.mju_negQuat(site_quat_conj, site_quat)
        # mujoco.mju_mulQuat(error_quat, target_quat, site_quat_conj)
        # mujoco.mju_quat2Vel(error_rot, error_quat, 1.0)
    end

    return error
end

function IK_inequality_constraints(params::NamedTuple, x::Vector)::Vector
    return zeros(eltype(x), 0)
end


IK_inequality_constraints (generic function with 1 method)

In [6]:
include(joinpath(@__DIR__, "utils","fmincon.jl"))

fmincon (generic function with 1 method)

In [7]:
function solve_multifinger_IK(;verbose=true)
    # instantiate model and data
    # model = load_model("models/scene_right_piano_hand.xml")
    model = load_model("/home/workstation4/daniel/ocrl-piano/Julia_fmincon_tests/piano_with_one_shadow_hand/scene.xml")
    data = init_data(model)
    reset!(model, data)

    # set the neutral pose
    neutral_joint_angles = vec(deepcopy(data.qpos))
    # neutral_joint_angles = convert(Vector{Float64}, neutral_joint_angles)
    
    # neutral_joint_angles = zeros(model.nq)
    @show typeof(neutral_joint_angles)

    # reset the model and data
    reset!(model, data)
    @show size(neutral_joint_angles)

    # LQR cost for neutral neutral_pose
    Q = 1e-3*diagm(ones(model.nq))
    @show size(Q)

    # fingertip sites and targets
    # fingertip_site_names = ["fingertip_index", "fingertip_middle", "fingertip_little"]
    # target_site_names = ["keymarker_1", "keymarker_2", "keymarker_3"]
    # fingertip_site_names = ["rh_shadow_hand/ffdistal_touch_site", "rh_shadow_hand/mfdistal_touch_site", "rh_shadow_hand/rfdistal_touch_site", "rh_shadow_hand/lfdistal_touch_site"]
    fingertip_site_names = ["rh_shadow_hand/mfdistal_touch_site", "rh_shadow_hand/ffdistal_touch_site"] 

    target_site_names = ["piano/white_key_site_70", "piano/white_key_site_72"]
    



    params = (Q=Q, model=model, data=data, fingertip_site_names, target_site_names, neutral_joint_angles)

    # primal bounds
    x_l = model.jnt_range[:,1]
    x_u = model.jnt_range[:,2]

    # inequality constraints
    c_l = zeros(0)
    c_u = zeros(0)



    # initial_guess 
    z0=0.001*rand(model.nq)

    # multifinger_IK_cost(params, z0)
    # IK_equality_constraints(params, z0)
    diff_type = :finite
    Z = fmincon(multifinger_IK_cost, IK_equality_constraints, IK_inequality_constraints, x_l, x_u, c_l, c_u, z0, params, diff_type; tol=1e-3, c_tol=1e-3, max_iters = 50, verbose=verbose)
    return Z
end

solve_multifinger_IK (generic function with 1 method)

In [8]:
qpos = solve_multifinger_IK()

# model = load_model("models/scene_right_piano_hand.xml")
model = load_model("/home/workstation4/daniel/ocrl-piano/Julia_fmincon_tests/piano_with_one_shadow_hand/scene.xml")

data = init_data(model)
reset!(model, data)
data.qpos .= qpos


typeof(neutral_joint_angles) = Base.ReshapedArray{Float64, 1, Transpose{Float64, UnsafeArrays.UnsafeArray{Float64, 2}}, Tuple{Base.MultiplicativeInverses.SignedMultiplicativeInverse{Int64}}}
size(neutral_joint_angles) = (114,)
size(Q) = (114, 114)
---------checking dimensions of everything----------
---------all dimensions good------------------------
---------diff type set to :finite (FiniteDiff.jl)---
---------testing objective gradient-----------------
---------testing constraint Jacobian----------------
---------successfully compiled both derivatives-----
---------IPOPT beginning solve----------------------

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
**************************************************************************

114×1 transpose(::UnsafeArrays.UnsafeArray{Float64, 2}) with eltype Float64:
 0.03287036696866052
 0.04446646272488679
 0.03356608047312797
 0.03356608047312797
 0.04446646272488679
 0.03356608047312797
 0.04446646272488679
 0.03356608047312797
 0.03356608047312797
 0.044466052993074656
 ⋮
 1.837928428594105e-5
 0.3594825267629504
 0.5017996600595444
 0.5017996600595444
 0.0002739662268638624
 0.458354725942736
 3.1301140166912254e-7
 0.00020339702268362748
 0.3592359415181507

In [9]:
visualise!(model, data)

 __  __            _        _____       _ _ 
|  \/  |          | |      / ____|     (_) |
| \  / |_   _     | | ___ | |     ___   _| |
| |\/| | | | |_   | |/ _ \| |    / _ \ | | |
| |  | | |_| | |__| | (_) | |___| (_) || | |
|_|  |_|\__,_|\____/ \___/ \_____\___(_) |_|
                                      _/ |  
                                     |__/   

Press "F1" to show the help message.
Mouse click commands:
┌───────────────────────────┬─────────────────────────────┐
│          Command          │ Description                 │
├───────────────────────────┼─────────────────────────────┤
│        Left click         │ Rotate scene                │
│        Right click        │ Translate scene             │
│     Left double-click     │ Select body                 │
│    Right double-click     │ Point camera at body        │
│         Left+CTRL         │ Rotate selected body        │
│        Right+CTRL         │ Translate selected body     │
│ Right double-click + CTRL │ Fix camera